In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
import numpy as np

In [3]:
path_img_input="../Pre-processing/dataset/face_dataset/Aaron_Peirsol/Aaron_Peirsol1.jpg"
image = Image.open(path_img_input)
image_array = np.array(image)
image_array.shape
#1080x1080x3

(1080, 1080, 3)

In [ ]:

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)